In [ ]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia analizę algorytmu Tabu Search (TS) dla różnych
# konfiguracji parametrów. Dla każdej kombinacji wykonywane jest
# kilka powtórzeń, zapisywane są statystyki jakości oraz czas
# działania. Wyniki trafiają do pliku CSV.

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.tabu import solve_tsp
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat


# USTAWIENIA
TSP_FILES = ["Dane_TSP_76.xlsx"]

PARAM_GRID = {
    "max_iter": [100_000_000],
    "stop_no_improve": [100_000],
    "tabu_tenure": [25],
    "n_neighbors": [100],
    "neighborhood_type": ["two_opt"],
}

REPEATS = 3
results = []


# ROZGRZANIE (KOMPILACJA JIT JEŚLI WYSTĘPUJE)
print("Rozgrzewanie (kompilacja JIT jeśli dotyczy)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"max_iter": 5, "tabu_tenure": 3, "n_neighbors": 5})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for max_iter in PARAM_GRID["max_iter"]:
        for stop_no_improve in PARAM_GRID["stop_no_improve"]:
            for tabu_tenure in PARAM_GRID["tabu_tenure"]:
                for n_neighbors in PARAM_GRID["n_neighbors"]:
                    for neighborhood_type in PARAM_GRID["neighborhood_type"]:

                        params = {
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "tabu_tenure": tabu_tenure,
                            "n_neighbors": n_neighbors,
                            "neighborhood_type": neighborhood_type,
                        }

                        # multiprocessing — równoległe powtórzenia
                        with Pool(processes=cpu_count()) as pool:
                            parallel_jobs = [
                                (solve_tsp, D, params) for _ in range(REPEATS)
                            ]
                            results_parallel = pool.map(run_single_repeat, parallel_jobs)

                        costs = [c for c, _, _ in results_parallel]
                        runtimes = [t for _, _, t in results_parallel]

                        results.append({
                            "instance": tsp_file,
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "tabu_tenure": tabu_tenure,
                            "n_neighbors": n_neighbors,
                            "neighborhood_type": neighborhood_type,
                            "mean_cost": round(np.mean(costs), 3),
                            "min_cost": round(np.min(costs), 3),
                            "mean_runtime": np.mean(runtimes),
                        })


# ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

df = pd.DataFrame(results)
save_experiment_results(df, time_seconds=int(elapsed), subfolder="TS")


# RAPORT
print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

print("\nPodsumowanie wyników:")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean()
      .round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")

Warm-up Numba (kompilacja JIT)...
Kompilacja zakończona. Start eksperymentu...


Instancja: Dane_TSP_76.xlsx

Podsumowanie (pierwsze 20 wierszy):
        instance   T0    alpha  T_min  max_iter neighborhood_type  mean_cost  min_cost  mean_runtime
Dane_TSP_76.xlsx 5000 0.999999 0.0001  10000000           two_opt  99192.872 97919.087      4.681982

Podsumowanie wyników:
                   mean_cost   min_cost  mean_runtime
neighborhood_type                                    
two_opt            99192.872  97919.087         4.682

Najlepsze parametry: {'instance': 'Dane_TSP_76.xlsx', 'T0': 5000, 'alpha': 0.999999, 'T_min': 0.0001, 'max_iter': 10000000, 'neighborhood_type': 'two_opt', 'mean_cost': 99192.872, 'min_cost': 97919.087, 'mean_runtime': 4.681982425200113}
